In [2]:
import rasterio

In [4]:
!ls "/ssd/floatingObjects/data/biscay_20180419.tif"

accra_20181031.cpg	     neworleans_20200202.shp
accra_20181031.dbf	     neworleans_20200202.shx
accra_20181031.fix	     neworleans_20200202.tif
accra_20181031.prj	     panama_20190425.cpg
accra_20181031.shp	     panama_20190425.dbf
accra_20181031.shx	     panama_20190425.fix
accra_20181031.tif	     panama_20190425_l2a.tif
biscay_20180419.cpg	     panama_20190425.prj
biscay_20180419.dbf	     panama_20190425.shp
biscay_20180419.fix	     panama_20190425.shx
biscay_20180419_l2a.tif      panama_20190425.tif
biscay_20180419.prj	     riodejaneiro_20180504.cpg
biscay_20180419.shp	     riodejaneiro_20180504.dbf
biscay_20180419.shx	     riodejaneiro_20180504.fix
biscay_20180419.tif	     riodejaneiro_20180504_l2a.tif
danang_20181005.cpg	     riodejaneiro_20180504.prj
danang_20181005.dbf	     riodejaneiro_20180504.shp
danang_20181005.fix	     riodejaneiro_20180504.shx
danang_20181005.prj	     riodejaneiro_20180504.tif
danang_20181005.shp	     sandiego_20180804.cpg
danang_20181005.shx	     sandiego

In [45]:
from data import l2abands, l1cbands, allregions
import sys
import numpy as np
sys.path.append("../code")
from transforms import calculate_fdi, calculate_ndvi

import os
image_folder = "/ssd/floatingObjects/data/"
viz_folder = "/ssd/floatingObjects/viz"


In [38]:

def load_raster(tif):
    with rasterio.open(tif) as src:
        scene = src.read()
        if (scene.shape[0] == 13):  # is L1C Sentinel 2 data
            scene = scene[[l1cbands.index(b) for b in l2abands]]
        profile = src.profile
    return scene, profile



In [49]:
from tqdm import tqdm
os.makedirs(viz_folder, exist_ok=True)
for region in tqdm(allregions):
    l1cpath = os.path.join(image_folder, region + ".tif")
    l2apath = os.path.join(image_folder, region + "_l2a.tif")
    imagepath = l2apath if os.path.exists(l2apath) else l1cpath

    scene, profile = load_raster(imagepath)
    scene = scene.astype(np.float32)
    
    fdi = calculate_fdi(scene)
    ndvi = calculate_ndvi(scene)
    
    img, ext = os.path.basename(imagepath).split(".")
    
    writeprofile = profile
    writeprofile["dtype"] = "float32"
    writeprofile["count"] = 1

    with rasterio.open(os.path.join(viz_folder, img + "_ndvi."+ext), "w", **writeprofile) as dst:
        dst.write(np.expand_dims(ndvi,0).astype(np.float32))
        
    with rasterio.open(os.path.join(viz_folder, img + "_fdi."+ext), "w", **writeprofile) as dst:
        dst.write(np.expand_dims(fdi,0).astype(np.float32))

100%|██████████| 15/15 [02:44<00:00, 10.96s/it]


In [50]:

imagepath = "/ssd/floatingObjects/Plastic/Mytilini2018/Mytilini180607.tif"
viz_folder = "/ssd/floatingObjects/Plastic/Mytilini2018/"

scene, profile = load_raster(imagepath)
scene = scene.astype(np.float32)

fdi = calculate_fdi(scene)
ndvi = calculate_ndvi(scene)

img, ext = os.path.basename(imagepath).split(".")

writeprofile = profile
writeprofile["dtype"] = "float32"
writeprofile["count"] = 1

with rasterio.open(os.path.join(viz_folder, img + "_ndvi."+ext), "w", **writeprofile) as dst:
    dst.write(np.expand_dims(ndvi,0).astype(np.float32))

with rasterio.open(os.path.join(viz_folder, img + "_fdi."+ext), "w", **writeprofile) as dst:
    dst.write(np.expand_dims(fdi,0).astype(np.float32))

In [34]:
image_folder

'/ssd/floatingObjects/data/kolkata_20201115_l2a.tif'